In [2]:
import pandas as pd

In [3]:
%%time
foldseek_results = pd.read_table('../data3/interim/foldseek_model_search.txt', 
                                 names=['query','target','fident','alnlen','mismatch','gapopen','qstart','qend','tstart','tend','evalue','bits'])

CPU times: user 10min 55s, sys: 3min 21s, total: 14min 17s
Wall time: 14min 22s


In [4]:
len(foldseek_results)

493429336

**Query:** train gene

**Target:** test gene

In [5]:
model_seq_info = pd.read_parquet('../data3/interim/model_seq_info.pq')

In [6]:
foldseek_results.head()

,query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits
0,e6299e5b463b1a860311be0137f9556ccd0a34e0e23dd4...,e6299e5b463b1a860311be0137f9556ccd0a34e0e23dd4...,1.000,38,0,0,1,38,1,38,0.000007,211
1,e6299e5b463b1a860311be0137f9556ccd0a34e0e23dd4...,8875ada9c44162c3043da6f1f650c8edf122068d9b78d5...,0.527,36,17,0,3,38,26,61,0.014210,119
2,e6299e5b463b1a860311be0137f9556ccd0a34e0e23dd4...,fc6bb202b5fb08d5fbda9efa8b02ffa248d111b6c05f13...,0.378,37,22,0,1,37,1,37,0.058260,102
3,e6299e5b463b1a860311be0137f9556ccd0a34e0e23dd4...,b18bceca9b63b1e33a0c894817104081350b055f18e742...,0.548,31,14,0,2,32,10,40,0.068780,100
4,e6299e5b463b1a860311be0137f9556ccd0a34e0e23dd4...,510529a206d0dcae2277bb78b352ed95f7a684dcb87c83...,0.466,30,15,0,3,32,25,54,0.068780,100


In [7]:
%%time
top_train_list = list()
for fold, fold_test_df in model_seq_info.groupby('test_fold'):
    print(fold)
    test_set_ids = set(fold_test_df['seq_id'])
    top_foldseek_df = (foldseek_results[~foldseek_results['query'].isin(test_set_ids) & 
                                        foldseek_results['target'].isin(test_set_ids)]
                     .sort_values('bits', ascending=False)
                     .groupby('target')
                     .head(1))
    top_train_list.append(top_foldseek_df)

0
1
2
3
4
CPU times: user 9min 48s, sys: 1min 56s, total: 11min 45s
Wall time: 11min 45s


In [8]:
cat_top_hit = pd.concat(top_train_list)

In [9]:
merged_cat_top_hit = (cat_top_hit[['query', 'target', 'bits']]
                      .merge(model_seq_info[['seq_id', 'defensive']]
                             .rename(columns={'seq_id': 'query', 
                                              'defensive': 'query_defensive'}), 
                             how='inner', on='query'))
merged_cat_top_hit['direction'] = merged_cat_top_hit['query_defensive'] * 2 - 1
merged_cat_top_hit['prediction'] = merged_cat_top_hit['bits'] * merged_cat_top_hit['direction']

In [10]:
out_df = (model_seq_info[['seq_id']]
          .merge(merged_cat_top_hit[['target', 'prediction']]
                 .rename(columns={'target': 'seq_id'}), 
                 how='left', on='seq_id'))
out_df['prediction'] = out_df['prediction'].fillna(0)
out_df['method'] = 'Foldseek/ProstT5 nearest neighbor'

In [11]:
len(out_df)

200458

In [12]:
out_df.to_parquet('../data3/interim/cv_predictions_foldseek.pq', index=False)